Pipeline이란, 데이터 변환 과정과 머신러닝 모델을 쉽게 연결해주는 역할을 하는 클래스.<br>
모델 체인의 좋은 예는, MinMaxScaler로 SVM 성능을 향상시키는 것.

In [4]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [7]:
cancer = load_breast_cancer()

x_train, x_test, y_train, y_test = train_test_split(
cancer.data, cancer.target, random_state=42)
scaler = MinMaxScaler().fit(x_train)

In [8]:
x_train_scaled = scaler.transform(x_train)

svm = SVC()

svm.fit(x_train_scaled, y_train)

x_test_scaled = scaler.transform(x_test)

print('Test Score : {:.2f}'.format(svm.score(x_test_scaled, y_test)))

Test Score : 0.97


/Users/kyle/kyle/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


GridSearchCV로 좋은 파라미터 찾는 알고리즘 추가하기

In [10]:
from sklearn.model_selection import GridSearchCV

In [13]:
param_grid = {'C':[0.001, 0.01, 0.1, 1, 10, 100],
             'gamma':[0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(svm, param_grid, cv=5)
grid.fit(x_train_scaled, y_train)

print('Best cv score : {:.2f}'.format(grid.best_score_))
print('Test score : {:.2f}'.format(grid.score(x_test_scaled, y_test)))
print('Best params : {}'.format(grid.best_params_))

Best cv score : 0.98
Test score : 0.98
Best params : {'C': 1, 'gamma': 1}


이 코드에서는 함정이 있는게 train set의 스케일링에 train set 전부를 사용한 문제가 있음. 이런 경우 cv할 때, train set에서 validation set을 분리하더라도 unseen data라는 가정의 validation set에 train set의 정보가 포함된다.

이럴 때는 cv를 위한 분할이 전처리 과정보다 먼저 일어나야 한다. 이럴 떄 Pipeline 클래스를 사용하면 된다. 여러 처리 단계를 묶어주는 역할

In [14]:
from sklearn.pipeline import Pipeline

In [27]:
# MinMaxScaler로 스케일링하고, SVC를 학습하는 파이프라인 생성
pipe = Pipeline([('scaler', MinMaxScaler()), ('svm', SVC()) ])

In [28]:
pipe.fit(x_train, y_train)

/Users/kyle/kyle/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [29]:
print('Test score : {:.2f}'.format(pipe.score(x_test, y_test)))

Test score : 0.97


그리드 서치까지 적용해보자.

In [32]:
param_grid = {'svm__C' : [0.001, 0.01, 0.1, 1, 10, 100],
             'svm__gamma':[0.001, 0.01, 0.1, 1, 10, 100]}

In [33]:
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(x_train, y_train)
print('Best cv score : {:.2f}'.format(grid.best_score_))
print('Test score : {:.2f}'.format(grid.score(x_test, y_test)))
print('Best params : {}'.format(grid.best_params_))

Best cv score : 0.97
Test score : 0.99
Best params : {'svm__C': 10, 'svm__gamma': 0.1}


이전과의 차이점은 cv에서 각 fold에 scaling이 적용된 부분.

### Pipeline Interface
이번에는 특성 추출, 특성 선택, 스케일링, 분류 4개 단계 파이프라인을 만들어보자.

In [ ]:
#는 다음에 만들어보자.